Create a Model that Predicts Overall Value

In [55]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:

data = pd.read_csv('fifa21_train.csv')


# Rename Columns

In [3]:
data.columns = data.columns.str.lower().str.replace(' ','_')
data.columns = data.columns.str.lower().str.replace('/','_')

In [4]:
data = data.rename(columns={'bp':'best_position',
                            'team_&_contract':'team_contract', 
                            'w_f':'weak_foot', 'sm': 'skill_moves', 'a_w': 'attacking_work_rate', 'd_w': 'defensive_work_rate', 'ir': 'international_reputation', 'wf': 'weak_foot' })

In [5]:
data.to_csv('fifa_cleaned3.csv')

# Checking

In [ ]:
# data.head()
# data.columns.tolist()
# data1.dtypes.tolist()
# data.isna().count().tolist()
# data['height']

# Decisions

1. OVR = ATT + IR (Overal Rating equals attributes and international reputation).
2. For those columns with "+", we´re going to keep the second number. We think the first number can be calculated from the columns of atributes, although we don´t know exactly how.
2. We know some columns are the sum of other columns. But we decided to keep the most of the information for the model to understand the patterns.



# New DF

#### Divide Columns, Choose Right

In [6]:
data_pos = data.loc[:, 'ls':'gk']

for col in data_pos.columns:
    data_pos[[col, col + '_right']] = data_pos[col].str.split('+', expand=True)

# Convertir las columnas resultantes en numéricas
data_pos = data_pos.astype(int)

data_pos = data_pos.drop(data_pos.loc[:, 'ls':'gk'].columns, axis=1)

#### Drop Columns

In [7]:
data_att = data.loc[:, 'crossing':'gk_reflexes']
data_att = data_att.drop(['skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis=1)

#### Clean Star

In [8]:
icon = '★'

data['international_reputation'] = data['international_reputation'].str.replace(icon, '')
data['international_reputation'] = data['international_reputation'].str.replace(' ', '')

data['international_reputation'].astype('int')

0        1
1        2
2        2
3        1
4        1
        ..
11696    1
11697    1
11698    1
11699    3
11700    1
Name: international_reputation, Length: 11701, dtype: int32

#### Categorical to Numerical

In [9]:
data_cat = pd.DataFrame(data['best_position'])

encoder = OneHotEncoder(drop='first').fit(data_cat)
encoded = encoder.transform(data_cat).toarray()

data_cat = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

#### Clean NaN

In [ ]:
# data_att.describe()

In [10]:
for col in data_att.columns:
    data_att[col] = data_att[col].fillna(data_att[col].mean())

# data_att.describe()

#### Concatenate Numerical

In [11]:
data_num = pd.concat([data_att, data_pos, data['international_reputation']], axis=1)
# data_nor.isna().sum().tolist()

# Check Correlation

In [15]:
data1_target = pd.concat([data['ova'], data_num], axis=1)
data1_target
correlation_matrix = data1_target.corr()
correlation_matrix

,ova,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
ova,1.000000,0.390354,0.306632,0.304702,0.493152,0.360994,0.350262,0.399692,0.372304,0.479454,...,0.395181,0.382723,0.300412,0.343115,0.507220,0.317776,0.692826,0.283944,0.233859,0.204647
crossing,0.390354,1.000000,0.645621,0.435570,0.800162,0.674937,0.854544,0.831791,0.751382,0.740585,...,0.733978,0.457167,0.400966,0.778864,0.682732,0.629131,0.590896,0.417238,0.404626,0.384012
finishing,0.306632,0.645621,1.000000,0.455388,0.650934,0.888847,0.820629,0.760857,0.695429,0.485792,...,0.883367,0.240675,-0.068253,0.896535,0.694219,0.839377,0.560906,-0.033656,-0.073428,-0.114214
heading_accuracy,0.304702,0.435570,0.455388,1.000000,0.630159,0.489435,0.531864,0.413944,0.366814,0.478891,...,0.480324,0.685740,0.502371,0.519654,0.244248,0.537305,0.516048,0.535808,0.513173,0.484591
short_passing,0.493152,0.800162,0.650934,0.630159,1.000000,0.683227,0.839028,0.764904,0.719229,0.886005,...,0.752279,0.614463,0.529120,0.750957,0.707982,0.665128,0.716718,0.550348,0.531136,0.494871
volleys,0.360994,0.674937,0.888847,0.489435,0.683227,1.000000,0.801099,0.806048,0.744717,0.542144,...,0.870661,0.315840,0.024688,0.854714,0.695439,0.838622,0.617829,0.055293,0.016066,-0.023491
dribbling,0.350262,0.854544,0.820629,0.531864,0.839028,0.801099,1.000000,0.843133,0.744447,0.704079,...,0.841421,0.436740,0.263272,0.897500,0.723750,0.758656,0.631071,0.299085,0.276400,0.247352
curve,0.399692,0.831791,0.760857,0.413944,0.764904,0.806048,0.843133,1.000000,0.853190,0.693619,...,0.834919,0.387005,0.233342,0.815808,0.749601,0.745755,0.642115,0.252629,0.229478,0.198960
fk_accuracy,0.372304,0.751382,0.695429,0.366814,0.719229,0.744717,0.744447,0.853190,1.000000,0.689447,...,0.805765,0.371314,0.249051,0.724357,0.715557,0.730167,0.598287,0.251366,0.236422,0.202289
long_passing,0.479454,0.740585,0.485792,0.478891,0.886005,0.542144,0.704079,0.693619,0.689447,1.000000,...,0.649654,0.582172,0.590282,0.592434,0.699561,0.519776,0.670140,0.591015,0.584720,0.559732


In [16]:
cor_ova = correlation_matrix[(correlation_matrix['ova']>0.2) | (correlation_matrix['ova']<(-0.2))]
cor_ova

# We´re going to keep only those highly correlated.

cor_ova.T.columns

Index(['ova', 'crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
       'ball_control', 'agility', 'reactions', 'shot_power', 'jumping',
       'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
       'positioning', 'vision', 'penalties', 'composure', 'marking',
       'standing_tackle', 'sliding_tackle'],
      dtype='object')

In [17]:
data_num = data_num[['crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
       'ball_control', 'agility', 'reactions', 'shot_power', 'jumping',
       'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
       'positioning', 'vision', 'penalties', 'composure', 'marking',
       'standing_tackle', 'sliding_tackle']]
data_num


,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
0,54,47,43,70,44.000000,61,44.000000,55,63,63,...,45,54,52.0,62.0,68.000000,54,54.000000,49,56,43.000000
1,66,79,76,68,76.000000,83,78.000000,72,63,79,...,68,54,33.0,78.0,72.000000,76,70.000000,35,20,22.000000
2,73,76,34,78,75.000000,85,89.000000,91,74,85,...,78,75,26.0,80.0,78.000000,73,82.000000,23,29,28.000000
3,44,42,58,62,36.000000,54,41.000000,46,57,61,...,38,61,57.0,31.0,54.000000,54,48.000000,55,58,55.000000
4,49,37,61,68,34.000000,64,44.000000,45,61,66,...,43,66,60.0,55.0,64.000000,49,58.000000,58,61,66.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,62,24,51,59,28.000000,57,49.000000,29,54,59,...,25,63,59.0,51.0,54.000000,39,46.000000,59,64,61.000000
11697,12,14,13,29,16.000000,17,15.000000,19,27,18,...,15,30,22.0,15.0,15.000000,19,56.000000,12,16,13.000000
11698,64,66,51,64,41.000000,71,57.000000,38,58,67,...,45,27,25.0,66.0,58.000000,65,58.000000,37,12,13.000000
11699,64,66,51,73,45.067324,74,49.617839,63,72,75,...,62,64,61.0,63.0,55.439194,53,59.947732,55,58,45.827702


# Normalize

In [18]:
tra = MinMaxScaler().fit(data_num)
nor = tra.transform(data_num)

data_nor = pd.DataFrame(nor, columns = data_num.columns)

data_nor

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
0,0.545455,0.478261,0.431818,0.720930,0.465116,0.615385,0.444444,0.561798,0.642857,0.637363,...,0.455556,0.517241,0.522222,0.638298,0.682353,0.534884,0.500000,0.516854,0.595238,0.440476
1,0.681818,0.826087,0.806818,0.697674,0.837209,0.857143,0.822222,0.752809,0.642857,0.813187,...,0.711111,0.517241,0.311111,0.808511,0.729412,0.790698,0.690476,0.359551,0.166667,0.190476
2,0.761364,0.793478,0.329545,0.813953,0.825581,0.879121,0.944444,0.966292,0.773810,0.879121,...,0.822222,0.758621,0.233333,0.829787,0.800000,0.755814,0.833333,0.224719,0.273810,0.261905
3,0.431818,0.423913,0.602273,0.627907,0.372093,0.538462,0.411111,0.460674,0.571429,0.615385,...,0.377778,0.597701,0.577778,0.308511,0.517647,0.534884,0.428571,0.584270,0.619048,0.583333
4,0.488636,0.369565,0.636364,0.697674,0.348837,0.648352,0.444444,0.449438,0.619048,0.670330,...,0.433333,0.655172,0.611111,0.563830,0.635294,0.476744,0.547619,0.617978,0.654762,0.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.636364,0.228261,0.522727,0.593023,0.279070,0.571429,0.500000,0.269663,0.535714,0.593407,...,0.233333,0.620690,0.600000,0.521277,0.517647,0.360465,0.404762,0.629213,0.690476,0.654762
11697,0.068182,0.119565,0.090909,0.244186,0.139535,0.131868,0.122222,0.157303,0.214286,0.142857,...,0.122222,0.241379,0.188889,0.138298,0.058824,0.127907,0.523810,0.101124,0.119048,0.083333
11698,0.659091,0.684783,0.522727,0.651163,0.430233,0.725275,0.588889,0.370787,0.583333,0.681319,...,0.455556,0.206897,0.222222,0.680851,0.564706,0.662791,0.547619,0.382022,0.071429,0.083333
11699,0.659091,0.684783,0.522727,0.755814,0.477527,0.758242,0.506865,0.651685,0.750000,0.769231,...,0.644444,0.632184,0.622222,0.648936,0.534579,0.523256,0.570806,0.584270,0.619048,0.474139


# Concatenate

In [20]:
data1 = pd.concat([data_cat, data_nor], axis=1)
# data1.describe().T
# data1.dtypes

# Train-Test Split

In [50]:
X = pd.concat([data1], axis=1)
y = data['ova']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Linear Regression

In [51]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [52]:
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.8761578098582261

# R2

In [53]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)

0.8679152985421951

# MAE

In [56]:
mae = mean_absolute_error(y_test, predictions_test)
mae

1.8531674102733613

# MSE

In [57]:
mse=mean_squared_error(y_test,predictions_test)
mse

6.035514693607758

# RMSE

In [58]:
rmse = np.sqrt(mean_squared_error(y_test,predictions_test))
rmse

2.4567284533720364

# Create a DEF Function

In [ ]:
def clean (df):

    # Rename columns
    df.columns = df.columns.str.lower().str.replace(' ','_')
    df.columns = df.columns.str.lower().str.replace('/','_')
    df = df.rename(columns={'bp':'best_position',
                                'team_&_contract':'team_contract', 
                                'w_f':'weak_foot', 'sm': 'skill_moves', 'a_w': 'attacking_work_rate', 'd_w': 'defensive_work_rate', 'ir': 'international_reputation', 'wf': 'weak_foot' })
    # Edit positions
    data_pos = df.loc[:, 'ls':'gk']
    for col in data_pos.columns:
        data_pos[[col, col + '_right']] = data_pos[col].str.split('+', expand=True)
    data_pos = data_pos.astype(int)
    data_pos = data_pos.drop(data_pos.loc[:, 'ls':'gk'].columns, axis=1)
    
    # Edit attributes
    data_att = df.loc[:, 'crossing':'gk_reflexes']
    data_att = data_att.drop(['skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis=1)
    
    # Edit international reputation
    icon = '★'
    df['international_reputation'] = df['international_reputation'].str.replace(icon, '')
    df['international_reputation'] = df['international_reputation'].str.replace(' ', '')
    df['international_reputation'].astype('int')
    
    # Edit best position
    data_cat = pd.DataFrame(df['best_position'])
    encoder = OneHotEncoder(drop='first').fit(data_cat)
    encoded = encoder.transform(data_cat).toarray()
    data_cat = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())
    
    # Fill na
    for col in data_att.columns:
        data_att[col] = data_att[col].fillna(data_att[col].mean())
    
    # Concat
    data_num = pd.concat([data_att, data_pos, df['international_reputation']], axis=1)
    
    # Choose num columns
    data_num = data_num[['crossing', 'finishing', 'heading_accuracy', 'short_passing',
           'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
           'ball_control', 'agility', 'reactions', 'shot_power', 'jumping',
           'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
           'positioning', 'vision', 'penalties', 'composure', 'marking',
           'standing_tackle', 'sliding_tackle']]
    
    # Normalise
    tra = MinMaxScaler().fit(data_num)
    nor = tra.transform(data_num)
    data_nor = pd.DataFrame(nor, columns = data_num.columns)
    
    # concatenate final
    df = pd.concat([data_cat, data_nor], axis=1)
    
    return df